In [ ]:
from grape import grape
from grape import algorithms
import re
from ast import literal_eval
import warnings

from os import path
import pandas as pd
import numpy as np
from deap import creator, base, tools
import random

from scripts.simulations.util import falsifier, evaluate
import multiprocessing

import os 
import mlflow

def evaluate_(ind, dummy):

    return 10,



# os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"

# # Set our tracking server uri for logging
# mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")


# warnings.filterwarnings('ignore')

GRAMMAR_FILE = 'test.bnf'
BNF_GRAMMAR = grape.Grammar(r"./scripts/scenarios/" + GRAMMAR_FILE)

In [2]:
# dir(BNF_GRAMMAR)

In [ ]:
BNF_GRAMMAR.n_rules

In [32]:
text = \
"""<rule>                 ::= Ensure safety of the pedestrian and the pet in a scenario where a pedestrian of <pedestrian-ethnicity> <pedestrian-gender> of age <pedestrian-age> <pedestrian-behavior> on the sidewalk trying to <pedestrian-behavior> with a <pet-color> <pet-breed> in the <terrain> during <lighting-condition> on a <weather> day
<pedestrian-ethnicity> ::= Asian | Europian | African-American
<pedestrian-gender>    ::= Male | Female | Others
<pedestrian-age>       ::= below-10 | 10-20 | 20-40 | above-40
<pedestrian-behavior>  ::= Walking | Jogging | Running | Cross
<pet-color>            ::= Black | White | Grey | Brown
<pet-breed>            ::= Dog | Cat | Possum | Rabbit
<terrain>              ::= City | Country | Town | Off-road
<lighting-condition>   ::= Morning | Noon | Evening | Twilight
<weather>              ::= Sunny | Windy | Cloudy | Snowy
"""

In [33]:
allowed_string = re.sub(r"[^a-zA-Z0-9:<>=\-\|\s]", "", text)
updated_string = re.sub(r"-", "_", allowed_string)

In [ ]:
print(updated_string)

## LARK grammar

In [2]:
from lark import Lark

# Define the grammar
grammar = r"""
start          : (goal)+
goal           : temporal_op "Achieve" objective "by" refined_goal "in scenario where" scenario
refined_goal   : "Achieving" objective "using" agent ("operationalized as" operation)? 
               | (refined_goal "followed by" refined_goal)* 
               | (refined_goal "parallely" refined_goal)*

operation      : task ("if" condition)? "performed by" module "taking input" input ("&" input)* "producing output" output ("&" output)*   
               | (operation "triggering" operation)* 
               | (operation "achieving" objective)* 

temporal_op    : STRING 

objective      : STRING
               | objective ("&" objective)*

condition      : STRING 
               | condition ("&" condition)*
               | condition ("|" condition)*

agent          : STRING
task           : STRING 
               | task ("&" task)*
module         : STRING
scenario       : STRING
input          : STRING
output         : STRING

%import common.ESCAPED_STRING   -> STRING
%import common.WS
%ignore WS
"""

# Instantiate the parser
parser = Lark(grammar, start="goal")

# # Define test inputs
# examples = [
    
#     """
#     ALWAYS Achieve "Safe Braking" 
#     by 
#         Achieving "Pedestrian Check" using "Perception Module"
#             operationalized as
#                 "capturing image" performed by "camera module" taking input "clock tick" producing output "image"
#                 triggering
#                 "Detect Pedestrian" performed by "pedestrian detection module" 
#                 taking input "image" producing output "pedestrian detection confidence" & "pedestrian detection flag"
#         parallely
#         Achieving "Sensing distance" using "Distance Estimation module" 
#             operationalized as
#                 "capturing point clouds" performed by "Lidar Module" taking input "clock tick" producing output "point cloud"
#                 triggering
#                 "estimate pedestrian distance" performed by "pixel to dist mapper module" 
#                 taking input "image" & "camera properties" & "point cloud" producing output "estimated position of pedestian" & "distance to pedestrian" & "safe braking distance"
#         followed by 
#         Achieving "braking" using "braking module"b
#             operationalized as 
#                 "Bring down throttle" & "Apply Brakes" if "pedestrian detection flag=True" & "distance to pedestrian <= safe braking distance" performed by "Braking Module" 
#                 taking input "pedestrian detection flag" & "distance to pedestrian" & "safe braking distance" producing output "Braking Status Flag" achieving "safe breaking"
#     in scenario where
#         "A boy is trying to cross street with a dog"
#     """
# ]

examples = [
    
    """
    "ALWAYS" Achieve "Safe Braking" 
    by 
        Achieving "Pedestrian Check" using "Perception Module"
            operationalized as
                "capturing image" performed by "camera module" taking input "clock tick" producing output "image"
                triggering
                "Detect Pedestrian" performed by "yolov5m" 
                taking input "image" producing output "pedestrian detection confidence" & "pedestrian detection flag"
        followed by 
        Achieving "braking" using "braking module"
            operationalized as 
                "Bring down throttle" & "Apply Brakes" if "pedestrian detection flag=True" performed by "Braking Module" 
                taking input "pedestrian detection flag" producing output "Braking Status Flag" achieving "safe breaking"
    in scenario where
        "A <pedestrian>  wearing a <dress> dress trying to cross road from <direction> at a distance of <distance> on a day with fog density <fog_density>"
    """
]

# Parse and print the results
for example in examples:
    print(f"Parsing: {example}")
    try:
        parse_tree = parser.parse(example)
        print(parse_tree.pretty())
    except Exception as e:
        print(f"Error: {e}")


Parsing: 
    "ALWAYS" Achieve "Safe Braking" 
    by 
        Achieving "Pedestrian Check" using "Perception Module"
            operationalized as
                "capturing image" performed by "camera module" taking input "clock tick" producing output "image"
                triggering
                "Detect Pedestrian" performed by "yolov5m" 
                taking input "image" producing output "pedestrian detection confidence" & "pedestrian detection flag"
        followed by 
        Achieving "braking" using "braking module"
            operationalized as 
                "Bring down throttle" & "Apply Brakes" if "pedestrian detection flag=True" performed by "Braking Module" 
                taking input "pedestrian detection flag" producing output "Braking Status Flag" achieving "safe breaking"
    in scenario where
        "A <pedestrian>  wearing a <dress> dress trying to cross road from <direction> at a distance of <distance> on a day with fog density <fog_density>"
    
g

In [59]:
from lark import Lark, Tree, Token

# Recursive function to find the "Detect Pedestrian" task and its model
def find_pedestrian_model(tree):
    if isinstance(tree, Tree):
        if tree.data == "operation":
            # Check if "Detect Pedestrian" is a task in this operation
            for i, child in enumerate(tree.children):
                if isinstance(child, Tree) and child.data == "task":
                    task_tokens = [str(t) for t in child.children if isinstance(t, Token)]
                    if "Detect Pedestrian" in task_tokens:
                        # Return the module (model) performing this task
                        module_index = i + 2  # "performed by" is followed by the module
                        if module_index < len(tree.children):
                            module_node = tree.children[module_index]
                            if isinstance(module_node, Token) and module_node.type == "STRING":
                                return module_node.value
    # Recurse into children
    for child in tree.children:
        result = find_pedestrian_model(child)
        if result:
            return result
    return None


# Extract the pedestrian detection model
model = find_pedestrian_model(parse_tree)
if model:
    print(f"The pedestrian detection model used is: {model}")
else:
    print("Pedestrian detection model not found.")

AttributeError: 'Token' object has no attribute 'children'

In [129]:
def get_perception_model(parse_tree):
    for _st in parse_tree.iter_subtrees():
        if isinstance(_st, Tree) and _st.data=='refined_goal':
            for _stt in _st.children:
                if isinstance(_stt, Tree) and _stt.data=='operation':
                    for _sttt in _stt.children:
                        if isinstance(_sttt, Tree)and _sttt.data=='operation':
                            task = _sttt.children[0]
                            module = _sttt.children[1]
                            try:
                                if task.children[0][1:-1] == "Detect Pedestrian":
                                    return module.children[0][1:-1]
                            except:
                                return None
                

In [131]:
get_perception_model(parse_tree=parse_tree)

'yolov5m'

## redisAI

In [2]:
# !pip install mlflow_redisai

In [22]:
import mlflow
from mlflow.deployments import get_deploy_client


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("yolo models")

target_uri = 'redisai://localhost:6379'  # host = localhost, port = 6379
redisai = get_deploy_client(target_uri)
redisai.create_deployment('yolov5s', "runs:/cd13bb44eb5e43249d35846711c1f8f3/yolov5s", config={'device': 'GPU'})

/home/darkaengl/Project/REAL/sim_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MlflowException: The specified model does not contain any of the supported flavors for deployment. The model contains the following flavors: dict_keys(['python_function', 'pytorch']). Supported flavors: ['torchscript', 'tensorflow']

In [13]:
dir(redisai)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'con',
 'create_deployment',
 'create_endpoint',
 'delete_deployment',
 'delete_endpoint',
 'explain',
 'get_deployment',
 'get_endpoint',
 'list_deployments',
 'list_endpoints',
 'predict',
 'predict_stream',
 'target_uri',
 'update_deployment',
 'update_endpoint']

In [20]:
# redisai.explain()

MlflowException: Computing model explanations is not yet supported for this deployment target

In [7]:
from redisai import Client as rai
import numpy as np
import cv2

model_path = '/home/darkaengl/Project/REAL/model/yolov5m.pt'

# Connect to Redis
redis_client = rai(host='localhost', port=6379)

# img = '/home/robb/Desktop/nn/YOLOv5_cars/yolov5/data/images/bus.jpg'
# img = cv2.imread(img)


# with open(model_path, 'rb') as f:
#     model = f.read()


# redis_client.modelstore('willitwork', 'torch', 'CPU', model)

# redis_client.tensorset('my_tensor', img)

# redis_client.modelexecute('willitwork', ['my_tensor'], ['output'])

# output_tensor = redis_client.tensorget('output')
# output_data = np.array(output_tensor)

In [11]:
# dir(redis_client)

In [10]:
redis_client.get_connection_kwargs()

{'db': 0,
 'username': None,
 'password': None,
 'socket_timeout': None,
 'encoding': 'utf-8',
 'encoding_errors': 'strict',
 'decode_responses': False,
 'retry_on_error': [],
 'retry': None,
 'health_check_interval': 0,
 'client_name': None,
 'redis_connect_func': None,
 'credential_provider': None,
 'host': 'localhost',
 'port': 6379,
 'socket_connect_timeout': None,
 'socket_keepalive': None,
 'socket_keepalive_options': None}

In [23]:
## Grammar

In [ ]:
<Goal> ::= "<Pattern>" <GoalName> ":" <Refinement> | <AgentAssignment> | <DomainConstraint>